Reducing the number of categories using zero-shot classification

# Importing Dataset

In [15]:
import os

# Changing directory to main directory for easy data access
working_directory = os.getenv("WORKING_DIRECTORY")
os.chdir(working_directory)

# Disabling error warning
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

In [16]:
import pandas as pd

# Loading is data previously cleaned data
path = 'data/books_cleaned.csv'
books = pd.read_csv(path)

# Printing the data to check validity
print(books.shape)
books.head(2)

(5197, 13)


,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,words_in_description,title_and_subtitle
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,199,Gilead
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,205,Spider's Web: A Novel


# Exploration and Preliminary Classification

In [17]:
books['categories'].value_counts().reset_index()

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
...,...,...
474,Aged women,1
475,Imperialism,1
476,Human-animal relationships,1
477,Amish,1


In [18]:
books["categories"].value_counts().reset_index().query("count > 50")

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
5,Religion,117
6,Philosophy,117
7,Comics & Graphic Novels,116
8,Drama,86
9,Juvenile Nonfiction,57


In [19]:
books[books["categories"] == 'Juvenile Fiction'].head(5)

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,words_in_description,title_and_subtitle
30,9780006646006,000664600X,Ocean Star Express,Mark Haddon;Peter Sutton,Juvenile Fiction,http://books.google.com/books/content?id=I2QZA...,Joe and his parents are enjoying a summer holi...,2002.0,3.50,32.0,1.0,129,Ocean Star Express
79,9780020442608,0020442602,The voyage of the Dawn Treader,Clive Staples Lewis,Juvenile Fiction,http://books.google.com/books/content?id=fDD3C...,"The ""Dawn Treader"" is the first ship Narnia ha...",1970.0,4.09,216.0,2869.0,65,The voyage of the Dawn Treader
85,9780030547744,0030547741,Where the Red Fern Grows,Wilson Rawls,Juvenile Fiction,http://books.google.com/books/content?id=IHpRw...,A young boy living in the Ozarks achieves his ...,2000.0,4.37,288.0,95.0,27,Where the Red Fern Grows: The Story of Two Dog...
86,9780060000141,0060000147,Poppy's Return,Avi,Juvenile Fiction,http://books.google.com/books/content?id=XbcMJ...,"There's trouble at Gray House, the girlhood ho...",2006.0,3.99,256.0,1086.0,82,Poppy's Return
87,9780060001537,0060001534,Diary of a Spider,Doreen Cronin,Juvenile Fiction,http://books.google.com/books/content?id=UWvZo...,This is the diary ... of a spider. But don't b...,2005.0,4.25,40.0,7903.0,93,Diary of a Spider


In [20]:
# Category mapping major categories into Fiction and Non-Fiction as well as Children version of them
# As such - Fiction/Non-Fiction/Children's Fiction/Children's Non-Fiction

category_mapping = {'Fiction' : "Fiction",
 'Juvenile Fiction': "Children's Fiction",
 'Biography & Autobiography': "Nonfiction",
 'History': "Nonfiction",
 'Literary Criticism': "Nonfiction",
 'Philosophy': "Nonfiction",
 'Religion': "Nonfiction",
 'Comics & Graphic Novels': "Fiction",
 'Drama': "Fiction",
 'Juvenile Nonfiction': "Children's Nonfiction",
 'Science': "Nonfiction",
 'Poetry': "Fiction"}

books_categories = books.copy()

books_categories["base_categories"] = books_categories["categories"].map(category_mapping)
books_categories.head(2)

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,words_in_description,title_and_subtitle,base_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,199,Gilead,Fiction
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,205,Spider's Web: A Novel,NaN


In [21]:
# Number of books categorized 
print(f'Number of Books Categorized: {books_categories[~(books_categories["base_categories"].isna())].shape[0]}')

# Books that still need to be categorized
print(f'Number of Books not Categorized: {books_categories[(books_categories["base_categories"].isna())].shape[0]}')


Number of Books Categorized: 3743
Number of Books not Categorized: 1454


# Zero-shot Classification

In [22]:
from transformers import pipeline

# Using transformers to classify non-Categorized books into the adult classes 
categories = ["Fiction", "Nonfiction"]

# Using Bart Large mnli
pipe = pipeline(task="zero-shot-classification",
                model="facebook/bart-large-mnli")

Device set to use cpu


In [30]:
data = books_categories.loc[books_categories["base_categories"] == "Fiction", "description"].reset_index(drop=True)[0]
pipe(data, categories)

{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [ ]:
import numpy as np

# Function to generate predictions using descriptions
# and returing "Fiction" or "Nonfiction"
def gen_pred(sequence, categories=categories, model=pipe):
    pred = model(sequence, categories)
    max_idx = np.argmax(pred["scores"])
    return pred["labels"][max_idx]

# Function to test the accuracy of `gen_pred` function
def test_gen_pred(category, size, df=books_categories):
    test_data = df.loc[df["base_categories"] == category, "description"].reset_index(drop=True)

    total_books = 0
    correct_preds = 0

    for idx in range(size):
        if gen_pred(test_data[idx]) == category:
            correct_preds += 1
        total_books += 1

    return correct_preds, total_books

In [ ]:
# Testing function `gen_pred` for the category "Fiction"
correct_preds_fic, total_books_fic = test_gen_pred(category="Fiction", size=300)

# Testing function `gen_pred` for the category "Nonfiction"
correct_preds_nonfic, total_books_nonfic = test_gen_pred(category="Nonfiction", size=300)

# Results
print(f'Fiction: {correct_preds_fic / total_books_fic}')
print(f'Nonfiction: {correct_preds_nonfic / total_books_nonfic}')
print(f'Combined: {(correct_preds_fic + correct_preds_nonfic) / (total_books_fic + total_books_nonfic)}')

Fiction: 0.68
Nonfiction: 0.8766666666666667
Fiction: 0.7783333333333333
